In [63]:
import requests, json
from pprint import pprint

In [64]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 10510885}


In [95]:
players = r['elements']

import pandas as pd
pd.set_option('display.max_columns', None)

In [66]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# create teams dataframe
teams = pd.json_normalize(r['teams'])

# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)


In [67]:
# # show result
# player_bio = df[['id','first_name', 'second_name','web_name', 'team_name', 'position_name', 'element_type']]
# team_names = player_bio['team_name'].unique().tolist()
# player_bio

In [69]:
# Write a function that obtains the gameweek history for a given player

# Modify to integrate optional argument for a specific gameweek in the past.

def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df


# show player #5's gameweek history
get_gameweek_history(5)

df.loc[5, ['web_name']]

web_name    Watkins
Name: 5, dtype: object

Potential modification to the *get_gameweek_history* function would be to add an optional parameter that represents the gameweek number $x$ to make it more robust. Note to handle errors. 

In [70]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
get_season_history(501).head(13)

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,2021/22,445044,60,63,99,1259,5,9,9,11,0,0,0,3,0,0,8,343,425.2,368.7,442.0,123.3,0,0.00,0.00,0.00,0.00
1,2022/23,445044,80,78,96,2062,2,7,8,40,0,0,0,2,0,0,5,310,410.8,639.4,514.0,156.0,23,2.68,4.76,7.44,30.15


In [71]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type', 'position_name']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Folarin,Balogun,Balogun,Arsenal,FWD
1,8,Gabriel,Fernando de Jesus,G.Jesus,Arsenal,FWD
2,13,Eddie,Nketiah,Nketiah,Arsenal,FWD
3,39,Keinan,Davis,Davis,Aston Villa,FWD
4,44,Jhon,Durán,Duran,Aston Villa,FWD


In [72]:
from tqdm.auto import tqdm
tqdm.pandas()

In [82]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

100%|██████████| 771/771 [06:24<00:00,  2.01it/s]


In [90]:
# get top scoring players by gameweek 19
halfway_leaders = points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
)

,element,web_name,total_points,goals_scored,assists
307,308,Salah,140,12,7
515,516,Son,127,11,5
59,60,Watkins,112,9,9
354,355,Haaland,112,14,5
525,526,Bowen,108,11,2
18,19,Saka,100,5,8
289,290,Alexander-Arnold,97,2,4
84,85,Solanke,96,12,1
556,557,Hee Chan,94,10,2
411,412,Gordon,92,6,6


In [93]:
points

,id_player,web_name,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,1,Balogun,1,2,16,0,True,2023-08-12T12:00:00Z,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,45,0,59090,0,0
1,1,Balogun,1,12,8,0,False,2023-08-21T19:00:00Z,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,45,-4744,63768,4959,9703
2,1,Balogun,1,21,10,0,True,2023-08-26T14:00:00Z,2,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,45,-7682,58109,2507,10189
3,1,Balogun,1,31,14,0,True,2023-09-03T15:30:00Z,3,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,44,-9696,49814,2558,12254
4,1,Balogun,1,43,9,0,False,2023-09-17T15:30:00Z,0,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,44,-12786,37048,0,12786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13709,571,Šarkić,571,149,6,0,True,2023-12-05T19:30:00Z,1,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,40,-510,32699,0,510
13710,571,Šarkić,571,160,16,0,True,2023-12-09T15:00:00Z,1,1,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,40,-411,32291,0,411
13711,571,Šarkić,571,170,19,0,False,2023-12-17T14:00:00Z,3,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,40,-523,31729,0,523
13712,571,Šarkić,571,180,7,0,True,2023-12-24T13:00:00Z,2,1,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,40,-408,31315,0,408
